# Data Wrangling

# 1. Introduction
We'll be using two data files(in Csv form), one contains personality scores for recruits and the other one contains the department they applied for.

# Getting the data I need in three steps:
1.Gather the data


2.Assess


3.Cleaning




# 1. Gather the data


Importing pandas and statistics modules

In [1]:
#import necessary packages
import pandas as pd
import statistics
import numpy as np

Reading personality and scores datasets into dataframes.

In [2]:
p_scores = pd.read_csv('personality_scores.csv', delimiter=';')

# 2. Assessing data

In [3]:
#number of samples and columns
p_scores.shape

(1555, 70)

#### Checking for duplicates

If there are any duplicates they'll be dropped

In [5]:
new_scores = p_scores.drop_duplicates(subset='ID', keep='first').dropna(axis=1)


#### checking if there are still any duplicates if any existed before we used .drop_duplicates

In [6]:
p_scores.duplicated().sum()

0

We don't have duplicates

### Setting the ID column as the index for our  dataframe.

In [7]:
new_scores = new_scores.set_index('ID')

In [8]:
new_scores.head()

,Section 5 of 6 [I am always prepared.],Section 5 of 6 [I am easily disturbed.],Section 5 of 6 [I am exacting (demanding) in my work.],Section 5 of 6 [I am full of ideas.],Section 5 of 6 [I am interested in people.],Section 5 of 6 [I am not interested in abstract ideas.],Section 5 of 6 [I am not interested in other people's problems.],Section 5 of 6 [I am not really interested in others.],Section 5 of 6 [I am quick to understand things.],Section 5 of 6 [I am quiet around strangers.],...,Section 5 of 6 [I often forget to put things back in their proper place],Section 5 of 6 [I pay attention to details.],Section 5 of 6 [I seldom feel blue (down).],Section 5 of 6 [I spend time reflecting on things.],Section 5 of 6 [I start conversations.],Section 5 of 6 [I sympathize with others' feelings.],Section 5 of 6 [I take time out for others.],Section 5 of 6 [I talk to a lot of different people at parties.],Section 5 of 6 [I use difficult words.],Section 5 of 6 [I worry about things.]
ID,,,,,,,,,,,,,,,,,,,,,
0,"(3, 5)","(4, 5)","(3, 5)","(5, 5)","(2, 3)","(5, 3)","(2, 3)","(2, 5)","(5, 5)","(1, 3)",...,"(3, 5)","(3, 5)","(4, 3)","(5, 5)","(1, 3)","(2, 5)","(2, 5)","(1, 3)","(5, 1)","(4, 3)"
1,"(3, 5)","(4, 5)","(3, 5)","(5, 5)","(2, 5)","(5, 3)","(2, 5)","(2, 5)","(5, 5)","(1, 3)",...,"(3, 5)","(3, 1)","(4, 1)","(5, 5)","(1, 5)","(2, 5)","(2, 5)","(1, 5)","(5, 3)","(4, 3)"
2,"(3, 5)","(4, 3)","(3, 3)","(5, 5)","(2, 5)","(5, 5)","(2, 5)","(2, 5)","(5, 5)","(1, 1)",...,"(3, 5)","(3, 5)","(4, 1)","(5, 3)","(1, 3)","(2, 5)","(2, 5)","(1, 3)","(5, 1)","(4, 3)"
3,"(3, 5)","(4, 5)","(3, 3)","(5, 5)","(2, 5)","(5, 3)","(2, 3)","(2, 3)","(5, 3)","(1, 3)",...,"(3, 1)","(3, 5)","(4, 1)","(5, 5)","(1, 5)","(2, 5)","(2, 5)","(1, 5)","(5, 1)","(4, 1)"
4,"(3, 3)","(4, 5)","(3, 3)","(5, 3)","(2, 3)","(5, 3)","(2, 3)","(2, 3)","(5, 5)","(1, 1)",...,"(3, 5)","(3, 5)","(4, 5)","(5, 5)","(1, 3)","(2, 3)","(2, 5)","(1, 3)","(5, 1)","(4, 3)"


In [9]:
#number of samples and columns
p_scores.shape

(1555, 70)

In [10]:
new_scores.shape

(1555, 50)

In [11]:
# the number of variables with nulls
nuls_columns = list(new_scores.isnull().sum()[new_scores.isnull().sum() > 0].index)
len(nuls_columns)

0

### 2. Write functions that will convert the answer on each personality test question and calculate the total scores for each of the subscales - apply unit testing.

Since we are working with string typed data , I'll be using the Eval function to try execute and interpret the string(argument)as python code. eval() evaluates the passed string as a Python expression and returns the result. For example, eval("1 + 1") interprets and executes the expression "1 + 1" and returns the result (2)

In [12]:
def sum_per_subscale(scores_per_person, subscale):
    subscale_sum = sum([eval(i)[1] for i in scores_per_person if eval(i)[0] == subscale])
    return(subscale_sum)
   
sum_per_subscale(new_scores.loc[0], 1)

30

### now we create a subscale list

In [13]:
subscale_list = ['1-Extraversion', '2-Agreeableness', '3-Conscientiousness', '4-EmotionalStability', '5-Intellect']
interp_list = ['extraversion_interpretation', 'agreeableness_interpretation', 'conscientiousness_interpretation', 'emotionalstability_interpretation', 'intellect_interpretation']

### we create a subscale_sum dataframe

In [14]:
subscale_sum_df = pd.DataFrame()
for i, subscale in enumerate(subscale_list):
    subscale_sum_df[subscale] = new_scores.apply(lambda row: sum_per_subscale(row, i+1), axis=1)     

### Our new dataframe looks like this

In [15]:
subscale_sum_df.head(10)

,1-Extraversion,2-Agreeableness,3-Conscientiousness,4-EmotionalStability,5-Intellect
ID,,,,,
0,30,40,48,36,42
1,42,46,46,40,42
2,28,40,40,38,42
3,30,38,38,40,38
4,28,34,46,38,36
5,48,46,42,46,36
6,38,44,50,36,42
7,30,48,48,42,42
8,40,46,48,44,48


### 3.Create a function/functions that takes in subscale total, and for each subscale, creates a new column called “{subscale_name}_interpretation” (e.g., conscientiousness_interpretation) and applies the labels “low”, “medium”, or “high” according to a person’s score on that subscale. Cut-off scores corresponding to the low, medium and high groups which are given by:

#### high: => mean + std_deviation/2


#### medium: between mean - std_deviation/2 and mean + std_deviation/2  


#### low: <= mean - std_deviation/2

mean is the average of the numbers and Standard Deviation is a measure of how spread out numbers are.

In [16]:
#Function to calculate cutOffs
def calculate_cutoffs(subscale):
    mean_1 = np.mean(subscale_sum_df[subscale])
    std_1 = np.std(subscale_sum_df[subscale])
    range_1 = [mean_1 - std_1/2, mean_1 + std_1/2]
    return(range_1)

### cutoff dataframe

In [17]:
cutoff_df = pd.DataFrame()
for subscale in subscale_list:
    cutoff_df[subscale] = calculate_cutoffs(subscale)
    
cutoff_df.head()

,1-Extraversion,2-Agreeableness,3-Conscientiousness,4-EmotionalStability,5-Intellect
0,26.815581,39.507903,39.187778,32.606789,37.352072
1,34.889885,44.852226,45.110614,39.629867,42.820276


### function for interpretation according to a person’s score

In [30]:
def get_interpretation(val, subscale):
    if (val <= cutoff_df[subscale].loc[0]):
        interp = 'low'
    elif ((val > cutoff_df[subscale].loc[0]) & (val < cutoff_df[subscale].loc[1])):
        interp = 'medium'
    elif (val >= cutoff_df[subscale].loc[1]):
        interp = 'high'
    return(interp)

#Checking if our function works
get_interpretation(subscale_sum_df['1-Extraversion'].loc[1], '1-Extraversion')

'high'

### New dataframe in accordance to our interpretation function

In [19]:
interp_df = pd.DataFrame()
for i, subscale in enumerate(subscale_list):
    interp_df[interp_list[i]] = subscale_sum_df.apply(lambda row: get_interpretation(row[subscale], subscale), axis=1)
    
interp_df.head()

,extraversion_interpretation,agreeableness_interpretation,conscientiousness_interpretation,emotionalstability_interpretation,intellect_interpretation
ID,,,,,
0,medium,medium,high,medium,medium
1,high,high,high,high,medium
2,medium,medium,medium,medium,medium
3,medium,low,low,high,medium
4,medium,low,high,medium,low


### 4.Read in the data in departments.csv and merge this data frame with the personality score data frame, keeping all applicants within the department data frame. Assert that the newly created merged data frame has the same amount of rows as the department data frame, and the expected number of columns.

In [20]:
departments = pd.read_csv('departments.csv', delimiter=';')

In [21]:
new_department = departments.drop_duplicates(subset='ID', keep='first').dropna(axis=1)
new_department.head()

,ID,Department
0,0,Data
1,1,Data
2,2,Data
3,3,Data
4,4,Data


In [22]:
dep = departments[['ID','Department']]

To combine dataframes, you first want to make sure that your dataframes are compatible. You want to make sure that the column and row structures match and that the values are in the same units, etc.

##### Check Compatibility of Dataframes

An easy way to check compatibility are to review the output of .info() for each dataframe.

In [23]:
dep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1555 entries, 0 to 1554
Data columns (total 2 columns):
ID            1555 non-null int64
Department    1555 non-null object
dtypes: int64(1), object(1)
memory usage: 24.4+ KB


In [24]:
subscale_sum_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1555 entries, 0 to 1554
Data columns (total 5 columns):
1-Extraversion          1555 non-null int64
2-Agreeableness         1555 non-null int64
3-Conscientiousness     1555 non-null int64
4-EmotionalStability    1555 non-null int64
5-Intellect             1555 non-null int64
dtypes: int64(5)
memory usage: 112.9 KB


In [25]:
interp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1555 entries, 0 to 1554
Data columns (total 5 columns):
extraversion_interpretation          1555 non-null object
agreeableness_interpretation         1555 non-null object
conscientiousness_interpretation     1555 non-null object
emotionalstability_interpretation    1555 non-null object
intellect_interpretation             1555 non-null object
dtypes: object(5)
memory usage: 72.9+ KB


#### The dataframes all have the same number of rows, thus we can easily merge them

In [26]:
scores = pd.concat([dep, new_scores,subscale_sum_df,interp_df], axis=1)
scores.head()

,ID,Department,Section 5 of 6 [I am always prepared.],Section 5 of 6 [I am easily disturbed.],Section 5 of 6 [I am exacting (demanding) in my work.],Section 5 of 6 [I am full of ideas.],Section 5 of 6 [I am interested in people.],Section 5 of 6 [I am not interested in abstract ideas.],Section 5 of 6 [I am not interested in other people's problems.],Section 5 of 6 [I am not really interested in others.],...,1-Extraversion,2-Agreeableness,3-Conscientiousness,4-EmotionalStability,5-Intellect,extraversion_interpretation,agreeableness_interpretation,conscientiousness_interpretation,emotionalstability_interpretation,intellect_interpretation
0,0,Data,"(3, 5)","(4, 5)","(3, 5)","(5, 5)","(2, 3)","(5, 3)","(2, 3)","(2, 5)",...,30,40,48,36,42,medium,medium,high,medium,medium
1,1,Data,"(3, 5)","(4, 5)","(3, 5)","(5, 5)","(2, 5)","(5, 3)","(2, 5)","(2, 5)",...,42,46,46,40,42,high,high,high,high,medium
2,2,Data,"(3, 5)","(4, 3)","(3, 3)","(5, 5)","(2, 5)","(5, 5)","(2, 5)","(2, 5)",...,28,40,40,38,42,medium,medium,medium,medium,medium
3,3,Data,"(3, 5)","(4, 5)","(3, 3)","(5, 5)","(2, 5)","(5, 3)","(2, 3)","(2, 3)",...,30,38,38,40,38,medium,low,low,high,medium
4,4,Data,"(3, 3)","(4, 5)","(3, 3)","(5, 3)","(2, 3)","(5, 3)","(2, 3)","(2, 3)",...,28,34,46,38,36,medium,low,high,medium,low


the newly created merged data frame has the same amount of rows as the department data frame, and the expected number of columns